# <span style="color:#4375c7">Financial Econometrics</span>
***
*The course material is for educational purposes only. Nothing herein should be taken as an investment advice or  offers any opinion  <br/> with respect to the suitability of any security. To obtain further information about this course, [contact us](http://wp.firrm.de/) or visit us during [office hours](https://finance.wiwi.tu-dortmund.de/professur/team/).*
***


# Hands on: Time Series Analysis II

In this part the knowledge from the last exercise will be extended and applied to real data. In addition, models assuming heteroscedasticity  model will be applied to calculate value at risk. 

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

### Core idea VAR
- Instead of just analyzing one variable using its past values (like classic AR models), the VAR model looks at two or more variables together.
- Each variable in the system is predicted using previous values of all variables, not just its own

**What does this mean:**
Suppose you want to model two economic indicators, like GDP and unemployment:
- With VAR, the current GDP depends on past values of GDP and unemployment.
- Likewise, the current unemployment rate depends on past values of unemployment and GDP.​

**Basic structure**
If you have two variables (let's call them $y_1$ and $y_2$), a simple VAR(1) model (with one lag) looks like this: 

$$
y_{1,t} = a_1 + b_{11} y_{1,t} + b_{12}y_{2,t} + e_{1,t}
y_{2,t} = a_1 + b_{21} y_{1,t} + b_{22}y_{2,t} + e_{2,t}
$$

- $y_{1,t}$ and $y_{2,t}$ are the current values
- The coefficients ($b_{ij}$) show how much each lagged variable conributes
- $e_{1,t}$ and $e_{2,t}$ are random errors or "shocks" which should not be predictable

**Difference to autoregressive models with exogenoues variables:**

*VAR Model (Vector Autoregression)*
- All variables are endogenous, meaning each variable in the system is modeled as depending on past values of itself and the other variables in the system.
- Example: If GDP and Inflation are both in the model, both are allowed to be influenced by their own past and each other's past.

*ARX or VARX Model (VAR with Exogenous Variables)*
- Includes exogenous variables (external, independent variables) that can affect the main variables, but are not themselves predicted by the system.
- Exogenous variables are added to the VAR framework to explain (potentially) external influences—these do not receive feedback from the endogenous system.

*Quick Analogy*

- VAR: All players influence each other, and everyone updates based on everyone else’s past actions.
- VARX: Like above, but now you add outside weather reports that influence the players, though the players' actions don’t influence the weather. Weather is exogenous.

## Exercise 1: VAR Model

Use the data files from the previous lecture (`sp500.csv` and `stock_indices.csv`)

1. Import the S&P 500 stock market index and the Goldman Sachs stock price. Select prices between 2018-01-01 and 2018-12-31. Visualize both time series.

2. Test wether both time series are stationary by using the Augmented Dickey Fuller (ADF) test. If the time series of price are not stationary, calculate the first order difference, i.e. log returns and check for stationarity again.

Another statistical property of multiple time series is <b>Cointegration</b>. In generic terms, this means that there is a long-term relationship between the variables [2, p. 254]. 

3. Apply the [Augmented Engle Granger](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.coint.html) test to check if both time series are cointegrated. Interpret the result.

4. Create a VAR model and fit it using both time series. Use `maxlags = 5`. Print the fitted model's summary. 

5. Forecast both time series 15 days into the future using the fitted VAR model. Visualize the forecast (use `.plot_forecast(...)`).

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss, coint
from statsmodels.tsa.api import VAR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
from scipy.stats import norm


In [ ]:
df_sp = pd.read_csv('TUdortmund/FinEcon/data/sp500.csv', index_col=0, parse_dates=True)
df_stock = pd.read_csv('TUdortmund/FinEcon/data/stock_indices.csv', index_col=0, parse_dates=True)

In [ ]:
df_sp = df_sp[(df_sp.index >= '2018-01-01') & (df_sp.index <= '2018-12-31')]
df_stock = df_stock[(df_stock.index >= '2018-01-01') & (df_stock.index <= '2018-12-31')]

In [ ]:
print(df_sp.columns)

In [ ]:
# Select GS (Goldman Sachs) from df_sp and SPX (S&P 500) from df_stock for 2018
series = pd.DataFrame({
    'GS': df_sp['GS'],
    'SPX': df_stock['SPX']
}).dropna()

ax = series.plot(figsize=(10, 5), title='S&P 500 (SPX) vs Goldman Sachs (GS) — 2018')
ax.set_xlabel('Date')
ax.set_ylabel('Price (USD)')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# ADF test on price series (levels)
results_GS = adfuller(series['GS'])
results_SPX = adfuller(series['SPX'])

def print_adf(res, name):
    print(f"ADF test for {name}:")
    print(f"  Test statistic: {res[0]:.4f}")
    print(f"  p-value: {res[1]:.4f}")
    print(f"  # lags used: {res[2]}")
    print(f"  # obs used: {res[3]}")
    print("  Critical values:")
    for k,v in res[4].items():
        print(f"    {k}: {v:.4f}")
    print()

print_adf(results_GS, 'GS (price level)')
print_adf(results_SPX, 'SPX (price level)')

$p-value > 0.05 we fail to reject the null hypothesis.

In [ ]:
# Compute log returns and re-run ADF on returns
rets = np.log(series).diff().dropna()

# ADF tests on log returns
res_GS_r = adfuller(rets['GS'])
res_SPX_r = adfuller(rets['SPX'])
print_adf(res_GS_r, 'GS (log returns)')
print_adf(res_SPX_r, 'SPX (log returns)')

# Quick visualization of returns
ax = rets.plot(figsize=(10,4), title='Log returns: GS and SPX (2018)')
ax.set_ylabel('Log return')
plt.tight_layout()
plt.show()

In [ ]:
coint_res = coint(series['GS'], series['SPX'])
print('Engle-Granger cointegration test (GS vs SPX):')
print(f'  Test statistic: {coint_res[0]:.4f}')
print(f'  p-value: {coint_res[1]:.4f}')
print('  Critical values (1%, 5%, 10%):')
for lvl in coint_res[2]:
    print(f'    {lvl:.4f}')

Interpretation: $p-value \geq 0.05$ -> cannot reject $H_0$ of no cointegration. No evidence of cointegration at conventional levels.


**Interpreting the Engle–Granger output**:
- The null hypothesis of the test is that the two series are not cointegrated. A small p-value (e.g. < 0.05) means we reject that null and conclude there is evidence of a long-run equilibrium relationship.
- If the p-value is large, we do not find evidence for cointegration; the series may wander independently in the long run.
- If you find cointegration, subsequent analysis commonly proceeds with an error-correction model (VECM) rather than a standard VAR.

In [ ]:
order = VAR(rets).select_order(5)
print('Lag order selection (lower is better):')
print(order)

model = VAR(rets)
res_var = model.fit(maxlags=5)

print(res_var.summary())

print(f'Number of lags used in fitted VAR: {res_var.k_ar}')

In [ ]:
steps = 15
lag_order = res_var.k_ar
last_obs = rets.values[-lag_order:]
fc = res_var.forecast(y=last_obs, steps=steps)

try:
    fc_index = pd.bdate_range(rets.index[-1], periods=steps+1)[1:]
except Exception:
    fc_index = pd.date_range(rets.index[-1], periods=steps+1, freq='D')[1:]

fc_df = pd.DataFrame(fc, index=fc_index, columns=rets.columns)
print('Forecast (log returns) — next {} periods:'.format(steps))
print(fc_df)

plot_df = pd.concat([rets, fc_df])
ax = plot_df.plot(figsize=(12,6), title=f'VAR forecast: historical returns and {steps}-step forecast')
ax.axvline(rets.index[-1], color='k', linestyle='--', label='Forecast start')
ax.set_xlabel('Date')
ax.set_ylabel('Log return')
plt.legend()
plt.tight_layout()
plt.show()

try:
    res_var.plot_forecast(steps, figsize=(10,6))
    plt.show()
except Exception as e:
    print('res_var.plot_forecast not available or failed:', e)

## ARCH and GARCH Models

1. What are ARCH and GARCH Models?
ARCH (Autoregressive Conditional Heteroskedasticity) and GARCH (Generalized ARCH) are models that allow the variance (volatility) of a time series to change over time—unlike standard models that assume constant volatility.

---

2. Why Use Them?
- Many financial series show **volatility clustering**: big changes tend to be followed by other big changes.
- ARCH/GARCH models help capture and forecast this dynamic behavior.

---

3. ARCH Model
ARCH(q): The variance at time \( t \) depends on the squared errors from the previous \( q \) periods.

$$
\epsilon_t = \sigma_t w_t \\
\sigma_t^2 = \alpha_0 + \alpha_1 \epsilon_{t-1}^2 + \cdots + \alpha_q \epsilon_{t-q}^2
$$

---

4. GARCH Model
GARCH(p, q): The variance depends on both earlier squared errors (like ARCH) **and** earlier variances.
$$
\sigma_t^2 = \alpha_0 + \sum_{i=1}^q \alpha_i \epsilon_{t-i}^2 + \sum_{j=1}^p \beta_j \sigma_{t-j}^2
$$




## Exercise 2: GARCH Model and Value at Risk

Install the `arch` package if necessary.

1. Visualize  the returns of Goldman Sachs in a plot as well as in a histogram.

2. Create a garch model using `arch_model` using the returns of Goldman Sachs. 

Hint: The estimation of the model parameter is done by maximum likelihood estimation. Small numbers can negatively influence the minimization process. Rescale your returns by a scalar, e.g. `100`.

3. Fit the model and print the summary.

4. Store the conditional volatility in an array `cond_vola`.

5. Calculate the value at risk for every $t$ using the conditional volatility for $\alpha = 0.05$ :
$$
\text{VaR}(t, \alpha) = \mu + \sigma({t|t-1})\,\Phi^{-1}(\alpha)
$$

$\Phi^{-1}$ is the inverse cumulative distribution function of a normal distribution. Use `scipy.stats.norm.ppf(alpha)`. $\mu$ is the mean value of the returns.

6. Visualize both the Value at Risk and the returns in a plot.

7. Calculate the VaR($\alpha$) based on historical returns, i.e. calculate the $\alpha$ quantile of the returns. Implement the result as a horizontal line in the plot. 

In [ ]:

gs_rets = rets['GS'].dropna()
print('Observations:', len(gs_rets))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

gs_rets.plot(ax=axes[0], title='GS Log Returns (2018)', color='C0')
axes[0].set_ylabel('Log return')
axes[0].axhline(0, color='k', linewidth=0.6)

axes[1].hist(gs_rets, bins=40, density=True, alpha=0.6, color='C1')
gs_rets.plot(kind='density', ax=axes[1], color='C2')
axes[1].set_title('Histogram and density: GS log returns')
axes[1].set_xlabel('Log return')

plt.tight_layout()
plt.show()

In [ ]:
am = arch_model(gs_rets * 100, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
res = am.fit(disp='off')
print(res.summary())

cond_vola = res.conditional_volatility / 100


In [ ]:
alpha = 0.05
gs_rets = gs_rets.dropna()
cond_vola = pd.Series(cond_vola, index=gs_rets.index)

z = norm.ppf(alpha)
mu = gs_rets.mean()
var_series = mu + cond_vola * z

rolling_hist_var = gs_rets.rolling(window=250, min_periods=1).quantile(alpha)
historical_var = gs_rets.quantile(alpha)

fig, ax = plt.subplots(figsize=(12,5))
gs_rets.plot(ax=ax, label='GS returns', color='C0', alpha=0.6)
var_series.plot(ax=ax, label=f'GARCH VaR (alpha={alpha})', color='C1')
rolling_hist_var.plot(ax=ax, label=f'Rolling hist. VaR (window=250, alpha={alpha})', color='C3')
ax.axhline(historical_var, color='k', linestyle='--', label=f'Historical VaR (alpha={alpha})')
ax.set_ylabel('Log return')
ax.set_title('GS returns and Value at Risk')
ax.legend()
plt.tight_layout()
plt.show()

### References
***
[1] Park, K. I., & Park. (2018). Fundamentals of Probability and Stochastic Processes with Applications to Communications.

[2] Brockwell, P. J., & Davis, R. A. (2016). Introduction to time series and forecasting.

[3] Montgomery, D. C., Jennings, C. L., & Kulahci, M. (2015). Introduction to time series analysis and forecasting. 

[4] Box, G. E., Jenkins, G. M., Reinsel, G. C., & Ljung, G. M. (2015). Time series analysis: forecasting and control. 

[5] Kelepouris, I., Kelepouris, D. Value at Risk estimation using GARCH model. [URL](https://www.kaggle.com/ionaskel/value-at-risk-estimation-using-garch-model) (visited: 2020-12-09).